In [ ]:
import matplotlib.pyplot as plt
import h5py
import rotation_mio as rot
import numpy as np
import barstrength2 as strng

h=0.732
G = 4.299e-6
a0=1
H0 = h*100
omega_lambda=0.716
omega_matter=0.1277/(h**2.)
omega0 = omega_lambda + omega_matter
vector2 = ('G1','G2','G3')
vector = ('00','01','03')

#path = 'home/ornela/SimCLUES/'
path = '/home/omarioni/'

atime = np.loadtxt(path + 'redshift_outputs.txt')
aexp = atime[:,2]
aexp = aexp[::-1]


path = '/mnt/is2/alejandro/ornella/'

snap = h5py.File(path + 'outputs_1/snap_496.h5py', 'r')

for i in range(0,3):
    cm   = snap['subhalo_0'+ str('%s' %vector[i])+ '/Center'].value
    r200 = snap['subhalo_0'+ str('%s' %vector[i])+ '/R200'].value

    pstr  = snap['subhalo_0'+ str('%s' %vector[i]) + '/Str/Coordinates'].value
    mstr  = snap['subhalo_0'+ str('%s' %vector[i]) + '/Str/Masses'].value
    vel   = snap['subhalo_0'+ str('%s' %vector[i])+ '/Str/Velocities'].value
    IDs   = snap['subhalo_0'+ str('%s' %vector[i]) + '/Str/Ids'].value
    tform = snap['subhalo_0'+ str('%s'%vector[i])+ '/Str/FormationTime'].value

    z = a0/aexp[0] - 1.
    Ht = H0*np.sqrt(omega_lambda+(1-omega0)*(1+z)**2+omega_matter*(1+z)**3)

#---aca paso las coordenadas respecto al centro de la galaxia------
    xstr = (pstr[:,0]-cm[0])*aexp[0]/h
    ystr = (pstr[:,1]-cm[1])*aexp[0]/h
    zstr = (pstr[:,2]-cm[2])*aexp[0]/h
    r = np.sqrt(xstr**2+ystr**2+zstr**2)

    v_x = vel[:,0] *np.sqrt(aexp[0]) + Ht *xstr/1000.
    v_y = vel[:,1] *np.sqrt(aexp[0]) + Ht *ystr/1000.
    v_z = vel[:,2] *np.sqrt(aexp[0]) + Ht *zstr/1000.

#----------------------masas----------------------------
    mstr = mstr/h

    r200 = r200*aexp[0]/h
    rgal=0.15*r200

    limit = np.where(r<rgal)
    r_sort = np.sort(r[limit])
    r_indice = np.argsort(r[limit])

    Mc_str = cumsum((mstr[limit])[r_indice])
    M_gal = Mc_str[-1]

#------------------ calculamos M90------------------------------------------
    razon = Mc_str/M_gal
    cincuenta, = np.where(razon < 0.5)

    r50 = r_sort[cincuenta][-1]             

#--------------------------------------------         
    veloc,=np.where(r<r50)

#----------componentes de la velocidad del centro de masa------------
    vxcm = sum(mstr[veloc]*v_x[veloc])/sum(mstr[veloc])
    vycm = sum(mstr[veloc]*v_y[veloc])/sum(mstr[veloc])
    vzcm = sum(mstr[veloc]*v_z[veloc])/sum(mstr[veloc])

#----- velocidades de las estrellas respecto del centro de masa de la galaxia---------
    vx = v_x - vxcm
    vy = v_y - vycm
    vz = v_z - vzcm

    e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z = rot.rot1(mstr,xstr,ystr,zstr,vx,vy,vz,3*aexp[0])

##posiciones de partículas que se quiere graficar
##como lo de arriba me da los versores hago las posiciones con esto 

    xn = e1x*xstr + e1y*ystr + e1z*zstr
    yn = e2x*xstr + e2y*ystr + e2z*zstr
    zn = e3x*xstr + e3y*ystr + e3z*zstr
    vxn = e1x*vx + e1y*vy + e1z*vz
    vyn = e2x*vx + e2y*vy + e2z*vz
    vzn = e3x*vx + e3y*vy + e3z*vz
    
    rn = np.sqrt(xn**2 + yn**2 + zn**2)
    
#     corte, = np.where(r < 3*aexp[0])

#     A2max, rmax, phimax = strng.a2max(mstr[corte], xn[corte], yn[corte], 20)

#     xx = xn*cos(phimax)+yn*sin(phimax)
#     yy = xn*-sin(phimax)+yn*cos(phimax)
#     zz = zn

    limit_str = np.where(rn < r200)
    
    pos=np.ndarray([np.size(xn[limit_str]),3])
    pos[:,0]=xn[limit_str]
    pos[:,1]=yn[limit_str]
    pos[:,2]=zn[limit_str]
    
    output_file.create_dataset(str('%03d'%vector[k])+'/aexp', data=aexp)
    output_file.create_dataset(str('%03d'%vector[k])+'/R200', data=r200)

    output_file.create_dataset('/subhalo_'+str('%03d'%vector[k])+'/Gas/Coordinates',    data=(h1.gas['pos'].in_units('kpc'))[kgas,:], dtype=np.float32)
    output_file.create_dataset('/subhalo_'+str('%03d'%vector[k])+'/Gas/Velocities',     data=vel_gas[kgas,:], dtype=np.float32)
    output_file.create_dataset('/subhalo_'+str('%03d'%vector[k])+'/Gas/Masses',         data=m_gas[kgas], dtype=np.float32)
#       output_file.create_dataset('/subhalo_'+str('%03d'% j)+'/Gas/Density',        data=rhogas[kgas], dtype=np.float32)
#       output_file.create_dataset('/subhalo_'+str('%03d'% j)+'/Gas/InternalEnergy', data=tempgas[kgas], dtype=np.float32)
#       output_file.create_dataset('/subhalo_'+str('%03d'% j)+'/Gas/Ids',            data=gas_id[kgas])

    output_file.create_dataset('/subhalo_'+str('%03d'%vector[k])+'/Drk/Coordinates',    data=(h1.dm['pos'].in_units('kpc'))[kdrk,:], dtype=np.float32)
    output_file.create_dataset('/subhalo_'+str('%03d'%vector[k])+'/Drk/Velocities',     data=vel_drk[kdrk,:], dtype=np.float32)
    output_file.create_dataset('/subhalo_'+str('%03d'%vector[k])+'/Drk/Masses',         data=m_drk[kdrk], dtype=np.float32)
#       output_file.create_dataset('/subhalo_'+str('%03d'% j)+'/Drk/Ids',            data=drk_id[kdrk])

    #if( (snap.npart[4] > 0) & (np.size(kstr) > 0) ):
    output_file.create_dataset('/subhalo_'+str('%03d'%vector[k])+'/Str/Coordinates',    data=pos, dtype=np.float32)
    output_file.create_dataset('/subhalo_'+str('%03d'%vector[k])+'/Str/Velocities',     data=vel_str[kstr,:], dtype=np.float32)
    output_file.create_dataset('/subhalo_'+str('%03d'%vector[k])+'/Str/Masses',         data=m_str[kstr], dtype=np.float32)
    output_file.create_dataset('/subhalo_'+str('%03d'%vector[k])+'/Str/FormationTime',  data=h1.star['tform'][kstr], dtype=np.float32)
    